In this notebook, you will learn how to make your first submission to the **[Tabular Playground Series - Jan 2021](https://admin.kaggle.com/c/tabular-playground-series-jan-2021/overview)** competition. 

# Make the most of this notebook!

You can use the "Copy and Edit" button in the upper right of the page to create your own copy of this notebook and experiment with different models. You can run it as-is and then see if you can make improvements.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
        
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')

# Read in the data files

In [ ]:
train = pd.read_csv(input_path / 'train.csv', index_col='id')
display(train.head())

In [ ]:
test = pd.read_csv(input_path / 'test.csv', index_col='id')
display(test.head())

In [ ]:
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

In [ ]:
submission.shape

# Let's try tabnet !

In [ ]:
!pip install pytorch-tabnet
import pandas as pd 
import numpy as np 
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

In [ ]:
train 

In [ ]:
features = train.columns[1:-1]
X      = train[features]
y      = np.log1p(train["target"])
X_test = test[features]
# y_true = solution["SalePrice"]

In [ ]:
X      = X.to_numpy()
y      = y.to_numpy().reshape(-1, 1)
X_test = X_test.to_numpy()

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    regressor = TabNetRegressor(verbose=1,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=1, max_epochs=2,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_test)))

predictions = np.mean(predictions_array,axis=0)

In [ ]:
predictions

In [ ]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )


In [ ]:
submission

In [ ]:
train

In [ ]:
submission.iloc[:,0:] = predictions

submission.to_csv('submission.csv')

In [ ]:
submission
